<a href="https://colab.research.google.com/github/yo20mom/CUK_Learning/blob/master/DeepLearningNLP/CNN_10%EC%A3%BC%EC%B0%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets import imdb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Conv1D, Dense, Embedding, MaxPool1D, Flatten
from tensorflow.keras.models import Model

In [ ]:
(x_train,y_train), (x_test,y_test) = imdb.load_data(num_words = 1000)

In [ ]:
x_train.shape
#0th sentence
print(x_train[0])
print(len(x_train[0]))

#1th sentence
print(len(x_train[1]))

In [ ]:
word_to_index = imdb.get_word_index()
word_to_index = {key:(value+3) for key,value in word_to_index.items()}
word_to_index["<PAD>"] = 0
word_to_index["<START>"] = 1
word_to_index["<UNK>"] = 2
word_to_index["<UNUSED>"] = 3

id_to_word = {value:key for key,value in word_to_index.items()}
print(' '.join(id_to_word[id] for id in x_train[0] ))

In [ ]:
imdb_index = imdb.get_word_index(path= 'imdb_word_index.json')
imdb_index = {k:(v+3) for k,v in imdb_index.items()}
imdb_index["<pad>"]= 0
imdb_index["<start>"]= 1
imdb_index["<unk>"]= 2
imdb_index["<unused>"]= 3

In [ ]:
imdb_index

In [ ]:
imdb_num2vec = {value:key for key,value in imdb_index.items()}

In [ ]:
sent = [imdb_num2vec[index] for index in x_train[0]]

In [ ]:
" ".join(sent)

In [ ]:
# pos / neg 개 수 얻기

print(pd.Series(y_train).value_counts())
print(pd.Series(y_test).value_counts())

In [ ]:
# 문장 하나당 단어가 얼마나 들어 있는지 확인

x_train_lengths = pd.DataFrame(x_train).applymap(len)
index, counts = np.unique(x_train_lengths, return_counts=True)

In [ ]:
plt.bar(index,counts)
print(max(index), min(index))
print(max(counts), min(counts))
x_train_lengths.describe()

In [ ]:
x_test_lengths = pd.DataFrame(x_test).applymap(len)
index, counts = np.unique(x_test_lengths, return_counts=True)

In [ ]:
plt.bar(index,counts)
print(max(index), min(index))
print(max(counts), min(counts))
x_test_lengths.describe()

In [ ]:
max_len = x_train_lengths.max().values
xtrain_padded = pad_sequences(x_train, maxlen= 300, padding= 'post')
xtest_padded = pad_sequences(x_test, maxlen= 300, padding= 'post')

In [ ]:
max(np.max(x_train, axis=None))

In [ ]:
#input

input = Input(shape=(300))

#embedding layer
layer = Embedding(input_dim= 1000, output_dim= 100)(input)

#Block 1

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = MaxPool1D()(layer)

#Block 2

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = MaxPool1D()(layer)

#Block 3

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = Conv1D(filters= 64, kernel_size=5, activation= 'relu', kernel_initializer='he_normal')(layer)

layer = MaxPool1D()(layer)

layer = Flatten()(layer)
layer = Dense(15,activation='tanh',kernel_initializer='he_normal')(layer)
layer = Dense(15,activation='tanh',kernel_initializer='he_normal')(layer)
layer = Dense(1,activation='sigmoid')(layer)

output = layer


In [ ]:
model = Model(input,output)
model.summary()

In [ ]:
model.compile('adam','binary_crossentropy', metrics=['acc'])

In [ ]:
model.fit(xtrain_padded,y_train, epochs=1, validation_data=(xtest_padded,y_test))